In [1]:
import pandas as pd
import polars as pl
from scipy.stats import beta

In [110]:
data = pl.read_csv("AI scores.txt", sep=" ")

data = data.with_columns(
    (pl.col("Won") + pl.col("Lost")).alias("Games_played")
    )

In [111]:
others = data.filter(pl.col("Games_played") < 5).sum().fill_null(
            pl.lit("Others"),
        )

data.extend(others)

# Remove players with less than 5 games played
data = data.filter(pl.col("Games_played") >= 5)

# Compute win rate
data = data.with_columns(
    (pl.col("Won") / pl.col("Games_played")).alias("Win_rate")
    )

In [5]:
def compute_confidence_interval(games_played, n_wins, confidence=0.9):
    print(games_played, n_wins)
    rv = beta(n_wins+1, games_played-n_wins+1)
    return rv.interval(confidence)


compute_confidence_interval(142, 142-57)

142 85


(0.5294325255044408, 0.6634709479547571)

In [118]:
data

Player,Won,Lost,Games_played,Win_rate
str,i64,i64,i64,f64
"""Me""",6,10,16,0.375
"""Daniel""",2,8,10,0.2
"""Giacomo""",4,10,14,0.285714
"""Simone""",19,20,39,0.487179
"""Silvio""",12,11,23,0.521739
"""Alessandro""",5,12,17,0.294118
"""Teresa""",6,7,13,0.461538
"""Others""",3,7,10,0.3


In [120]:
57 / 142

0.4014084507042254

In [117]:
data.sum()

Player,Won,Lost,Games_played,Win_rate
str,i64,i64,i64,f64
null,57,85,142,2.925289
